# Project1: Writing a Data Scientist Blog Post

## Seattle Airbnb open datasets  
Difficult one, waiting after getting the certificates ...  

## Business understanding
there are questions I wnat to explore in Seattle Airbnb open datasets  
1. What's is the distribution of airbnb in neighbourhood ? can we say the more airbnbs in neighbourhood, the airbnb prices are higher ?  
2. When is the popular seasons to visit Seattle ? how is the price changing ?  
3. How well can we predict an airbnb price? What variables correlate well with airbnb price ?  
4. Can you describe the vibe of each Boston neighborhood using listing descriptions?  
5. Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle? 
6.  if there is anything about the properties that helps you predict price ?
7.  Can you find negative and positive reviews based on the text?

# Data understanding  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import defaultdict

In [36]:
# reading raw data
'''
Listings, including full descriptions and average review score
Reviews, including unique id for each reviewer and detailed comments
Calendar, including listing id and the price and availability for that day
'''
df_Seattle_listing = pd.read_csv('./Seattle Airbnb Open Data/listings.csv')
df_Seattle_calendar = pd.read_csv('./Seattle Airbnb Open Data/calendar.csv')
df_Seattle_reviews = pd.read_csv('./Seattle Airbnb Open Data/reviews.csv')

In [37]:
pd.set_option('display.max_columns',500)
# df_Seattle_listing.head()

In [38]:
# dataframe are huge, drop columns which are usless for our business questions
# the columns relating urls are surely useless
df_Seattle_listing = df_Seattle_listing.drop(columns=['listing_url','scrape_id','zipcode','host_url','host_response_time',
                                                      'host_picture_url','host_verifications','host_thumbnail_url',
                                                     'host_has_profile_pic','host_identity_verified','latitude','longitude',
                                                     'thumbnail_url','medium_url','picture_url','xl_picture_url'])
df_Seattle_listing = df_Seattle_listing.drop(columns=['square_feet','experiences_offered','host_is_superhost','is_location_exact',
                                                     'require_guest_profile_picture','require_guest_phone_verification','requires_license',
                                                     'instant_bookable'])

In [46]:
df_Seattle_listing[df_Seattle_listing.id == 953595]

,id,last_scraped,name,summary,space,description,neighborhood_overview,notes,transit,host_id,host_name,host_since,host_location,host_about,host_response_rate,host_acceptance_rate,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,market,smart_location,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,cancellation_policy,calculated_host_listings_count,reviews_per_month
1,953595,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,98%,100%,Queen Anne,6.0,6.0,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,Seattle,"Seattle, WA",US,United States,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,WASHINGTON,strict,6,1.48


In [41]:
df_Seattle_calendar.columns, df_Seattle_calendar.shape

(Index(['listing_id', 'date', 'available', 'price'], dtype='object'),
 (1393570, 4))

In [47]:
df_Seattle_calendar[df_Seattle_calendar.listing_id == 953595]

,listing_id,date,available,price
365,953595,2016-01-04,t,$125.00
366,953595,2016-01-05,t,$125.00
367,953595,2016-01-06,f,NaN
368,953595,2016-01-07,f,NaN
369,953595,2016-01-08,f,NaN
...,...,...,...,...
725,953595,2016-12-29,t,$229.00
726,953595,2016-12-30,t,$294.00
727,953595,2016-12-31,t,$358.00
728,953595,2017-01-01,t,$439.00


In [42]:
df_Seattle_reviews.columns, df_Seattle_reviews.shape

(Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object'),
 (84849, 6))

In [48]:
df_Seattle_reviews[df_Seattle_reviews.listing_id == 953595]

,listing_id,id,date,reviewer_id,reviewer_name,comments
83447,953595,6643648,2013-08-19,7323285,Cherry,The reservation was canceled 33 days before ar...
83448,953595,6643745,2013-08-19,7557645,Georgianne,The reservation was canceled 38 days before ar...
83449,953595,8990789,2013-12-01,1586593,Jo Breen (Jody),This apartment in upper Queen Anne in Seattle ...
83450,953595,12389494,2014-05-01,1586593,Jo Breen (Jody),I have reviewed the Bright + Airy Queen Anne a...
83451,953595,17264483,2014-08-10,13647404,Brian,The Bright + Airy Queen Anne Apartment was per...
83452,953595,17823626,2014-08-18,15108582,Helen,"Fabulous apartment. Very clean, good supply of..."
83453,953595,18322321,2014-08-25,17931620,Ernesto,Andrea and her place were great! She and her p...
83454,953595,19090282,2014-09-06,11137841,Ashley,"Andrea, our host, wasn't in town but she commu..."
83455,953595,19390238,2014-09-11,9123455,Courtney,Andrea was a great host - very easy going and ...
83456,953595,26515505,2015-02-15,3243449,Mehmet Cenk,Best place to stay in Seattle.


In [13]:
df_Seattle = pd.merge(df_Seattle_calendar,df_Seattle_reviews,how='inner',on=['listing_id'],suffixes=('_calendar', '_reviews'))

In [14]:
df_Seattle.dropna().shape

(22485359, 9)

In [7]:
df_Seattle_listing = df_Seattle_listing.rename(columns={'id':'listing_id'})

In [8]:
df_Seattle = pd.merge(df_Seattle_listing,df_Seattle,on='listing_id')
df_Seattle.shape

(30969885, 100)

In [9]:
df_Seattle.dropna()

MemoryError: Unable to allocate 15.7 GiB for an array with shape (68, 30969885) and data type object